In [1]:
import h5py
import numpy as np
import time
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical

In [2]:
print(str(time.ctime()) + ": Initializing...")

sarsmerscov_train = h5py.File('/ccs/proj/gen150/arjun2612/ORNL/datasets/sars-mers-cov2_train.h5', 'r')
sarsmerscov_val = h5py.File('/ccs/proj/gen150/arjun2612/ORNL/datasets/sars-mers-cov2_val.h5', 'r')

lt = list(open('/gpfs/alpine/gen150/scratch/arjun2612/ORNL_Coding/Code/sars_mers_cov2_dataset/label_train.txt', 'r'))
lv = list(open('/gpfs/alpine/gen150/scratch/arjun2612/ORNL_Coding/Code/sars_mers_cov2_dataset/label_val.txt', 'r')) # open all files

cvae_embeddings = np.load('/gpfs/alpine/gen150/scratch/arjun2612/ORNL_Coding/Code/sars_mers_cov2_dataset/cvae/sars-mers-cov2-embeddings.npy', 'r')
cvae_samples = np.load('/gpfs/alpine/gen150/scratch/arjun2612/ORNL_Coding/Code/sars_mers_cov2_dataset/cvae/sars-mers-cov2-samples.npz', 'r')

print(str(time.ctime()) + ": Successfully loaded all data sets!")

Thu Jul  8 16:03:47 2021: Initializing...
Thu Jul  8 16:03:47 2021: Successfully loaded all data sets!


In [3]:
train_size = 60000
val_size = 15000

In [4]:
label_training = np.array([])
label_validation = np.array([])

for i in range(len(lt)):
    num = int(str(lt[i]).strip('\n'))
    label_training = np.append(label_training, num)
    
for j in range(len(lv)):
    num = int(str(lv[j]).strip('\n'))
    label_validation = np.append(label_validation, num)

In [5]:
trainset = np.array(sarsmerscov_train['contact_maps']).astype(float) # 60000 x 24 x 24 x 1
valset = np.array(sarsmerscov_val['contact_maps']).astype(float) # 15000 x 24 x 24 x 1

trainset, label_training = shuffle(trainset, label_training, random_state=0)
valset, label_validation = shuffle(valset, label_validation, random_state=0)

trainset = trainset[0:train_size]
valset = valset[0:val_size]
label_training = label_training[0:train_size]
label_validation = label_validation[0:val_size]

train_3D = np.tril(trainset[:, :, :, 0])
val_3D = np.tril(valset[:, :, :, 0])

lt_onehot = to_categorical(label_training) # make one hot vectors
lv_onehot = to_categorical(label_validation)

In [6]:
cvae_embeddings = np.squeeze(cvae_embeddings)[0:val_size]

In [7]:
lt = None
lv = None
sarsmerscov_train = None
sarsmerscov_val = None # garbage collection to free up memory

In [8]:
np.savez('smc2_dataset.npz', train4D=trainset, val4D=valset, train3D=train_3D, val3D=val_3D, ltoh=lt_onehot, lvoh=lv_onehot, labtrain=label_training, labval=label_validation)